另一种双均线策略

In [ ]:
import pandas as pd
from datetime import datetime
import trdb2py

isStaticImg = False
width = 960
height = 768

pd.options.display.max_columns = None
pd.options.display.max_rows = None

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

In [ ]:
# 具体基金
# asset = 'jrj.510310'
asset = 'jqdata.000036_XSHG|1d'

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

# 初始资金池
paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

# 卖出参数，全部卖出
paramssell = trdb2py.trading2_pb2.SellParams(
    perVolume=1,
)

In [ ]:
asset = 'jqdata.000036_XSHG|1d'

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='沪深300指数',
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

# 具体基金
# asset = 'jrj.510310'
asset = 'jqdata.000036_XSHG|1d'

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

lstema = [55, 120]

for ema in lstema:    
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[ema],
    )    
            
    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0, buy1])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='sma.{}'.format(ema),
    ))
 
lstpnl1 = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=2.3)

trdb2py.showPNLs(lstpnl1 + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
ret = trdb2py.getAssetCandles2(trdb2cfg, asset, tsStart, tsEnd, indicators=['sma.55', 'sma.120'])
# print(ret) 
# ret
trdb2py.showAssetCandles2('沪深300指数', ret, indicators=['sma.55', 'sma.120'], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

asset = 'jqdata.000036_XSHG|1d'

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))


for ema0 in range(55, 56):
    for ema1 in range(ema0 + 1, 241):
        buy0 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['upcross'],
            strVals=['ta-sma.{}'.format(ema1)],
        )

        buy1 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema0],
        )
        
        buy2 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['up'],
            strVals=['ta-sma.{}'.format(ema0)],
        )
        
        buy3 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema1],
        )        
        
        buy4 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['upcross'],
            strVals=['ta-sma.{}'.format(ema0)],
            group=1,
        )

        buy5 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema0],
            group=1,            
        )
        
        buy6 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['up'],
            strVals=['ta-sma.{}'.format(ema1)],
            group=1,            
        )
        
        buy7 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema1],
            group=1,            
        )                

        sell0 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['downcross'],
            strVals=['ta-sma.{}'.format(ema0)],
        )
        
        sell1 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['downcross'],
            strVals=['ta-sma.{}'.format(ema1)],
            group=1,                        
        )        
        
#         sell1 = trdb2py.trading2_pb2.CtrlCondition(
#             name='indicatorsp',
#             operators=['down'],
#             strVals=['ta-sma.{}'.format(ema0)],
#         )

        s0 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2asset(asset),
        )

        s0.buy.extend([buy0, buy1, buy2, buy3, buy4, buy5, buy6, buy7])
        s0.sell.extend([sell0, sell1])
        s0.paramsBuy.CopyFrom(paramsbuy)
        s0.paramsSell.CopyFrom(paramssell) 
        s0.paramsInit.CopyFrom(paramsinit)        
        lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2asset(asset)],
            startTs=tsStart,
            endTs=tsEnd,
            strategies=[s0],
            title='sma{}&sma{}'.format(ema0, ema1),
        ))
        
# len(lstparams)
lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=4)        

trdb2py.showPNLs(lstpnlmix + [pnlBaseline, lstpnl1[0]], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnl1 + lstpnlmix + [pnlBaseline])

# dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 2]

dfpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

In [ ]:
lstparams = []

asset = 'jqdata.000036_XSHG|1d'

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))


for ema0 in range(2, 241):
    for ema1 in range(ema0 + 1, 241):
        buy0 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['upcross'],
            strVals=['ta-sma.{}'.format(ema1)],
        )

        buy1 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema0],
        )
        
        buy2 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['up'],
            strVals=['ta-sma.{}'.format(ema0)],
        )
        
        buy3 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema1],
        )        
        
        buy4 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['upcross'],
            strVals=['ta-sma.{}'.format(ema0)],
            group=1,
        )

        buy5 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema0],
            group=1,            
        )
        
        buy6 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['up'],
            strVals=['ta-sma.{}'.format(ema1)],
            group=1,            
        )
        
        buy7 = trdb2py.trading2_pb2.CtrlCondition(
            name='waittostart',
            vals=[ema1],
            group=1,            
        )                

        sell0 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['downcross'],
            strVals=['ta-sma.{}'.format(ema0)],
        )
        
        sell1 = trdb2py.trading2_pb2.CtrlCondition(
            name='indicatorsp',
            operators=['downcross'],
            strVals=['ta-sma.{}'.format(ema1)],
            group=1,                        
        )        
        
#         sell1 = trdb2py.trading2_pb2.CtrlCondition(
#             name='indicatorsp',
#             operators=['down'],
#             strVals=['ta-sma.{}'.format(ema0)],
#         )

        s0 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2asset(asset),
        )

        s0.buy.extend([buy0, buy1, buy2, buy3, buy4, buy5, buy6, buy7])
        s0.sell.extend([sell0, sell1])
        s0.paramsBuy.CopyFrom(paramsbuy)
        s0.paramsSell.CopyFrom(paramssell) 
        s0.paramsInit.CopyFrom(paramsinit)        
        lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2asset(asset)],
            startTs=tsStart,
            endTs=tsEnd,
            strategies=[s0],
            title='sma{}&sma{}'.format(ema0, ema1),
        ))

# len(lstparams)        
lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=4)        

trdb2py.showPNLs(lstpnlmix + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmix + [pnlBaseline])

# dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 2]

dfpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

In [ ]:
lstparams = []

# 具体基金
# asset = 'jrj.510310'
asset = 'jqdata.000036_XSHG|1d'

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

lstema = [71, 60, 80, 97, 55, 115]

for ema in lstema:    
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[ema],
    )    
            
    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0, buy1])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='sma.{}'.format(ema),
    ))
 
lstpnlmix0 = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=2.3)

trdb2py.showPNLs(lstpnlmix0 + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

asset = 'jqdata.000036_XSHG|1d'

# 起始时间，0表示从最开始算起
tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

lstema0 = [71, 60, 80, 97, 55, 115]
lstema1 = [141, 141, 141, 141, 141, 141]


for i in range(0, len(lstema0)):
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ta-sma.{}'.format(lstema0[i])],
    )

    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[lstema0[i]],
    )

    buy2 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['up'],
        strVals=['ta-sma.{}'.format(lstema1[i])],
    )

    buy3 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[lstema1[i]],
    )        
    
    buy4 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ta-sma.{}'.format(lstema1[i])],
        group=1,
    )

    buy5 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[lstema0[i]],
        group=1,        
    )

    buy6 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['up'],
        strVals=['ta-sma.{}'.format(lstema0[i])],
        group=1,        
    )

    buy7 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[lstema1[i]],
        group=1,        
    )            

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ta-sma.{}'.format(lstema0[i])],
    )
    
    sell1 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ta-sma.{}'.format(lstema1[i])],
        group=1,                
    )    

#         sell1 = trdb2py.trading2_pb2.CtrlCondition(
#             name='indicatorsp',
#             operators=['down'],
#             strVals=['ta-sma.{}'.format(ema0)],
#         )

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0, buy1, buy2, buy3, buy4, buy5, buy6, buy7])
    s0.sell.extend([sell0, sell1])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='sma{}&sma{}'.format(lstema0[i], lstema1[i]),
    ))
        
lstpnlmix1 = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=3)        

trdb2py.showPNLs(lstpnlmix0 + lstpnlmix1 + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmix + lstpnlmix0 + [pnlBaseline])

# dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 2]

dfpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

In [ ]:
lstparams = []

lstsma = [71, 60, 80, 97, 55, 115]

# 具体基金
# asset = 'jrj.510310'
asset = 'jqdata.000036_XSHG|1d'

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

for ema in lstsma:    
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[ema],
    )    
            
    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0, buy1])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='sma.{}'.format(ema),
    ))
 
lstpnlmix1 = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=2.3)

# trdb2py.showPNLs(lstpnlmix + lstpnlmix1 + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
asset = 'jqdata.000036_XSHG|1d'

# 起始时间，0表示从最开始算起
tsStart = 0
# tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
# tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='沪深300指数',
)  

pnlBaselineF = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaselineF, toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

lstems = [71, 60, 80, 97, 55, 115, 141]
# 起始时间，0表示从最开始算起
tsStart = 0
# tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-12-31', '%Y-%m-%d'))

for ema in lstems:    
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[ema],
    )    
            
    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0, buy1])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='sma.{}'.format(ema),
    ))
 
lstpnlmixF0 = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=2.3)

trdb2py.showPNLs(lstpnlmixF0 + [pnlBaselineF], toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

asset = 'jqdata.000036_XSHG|1d'

# 起始时间，0表示从最开始算起
tsStart = 0
# tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-12-31', '%Y-%m-%d'))

lstema0 = [71, 60, 80, 97, 55, 115]
lstema1 = [141, 141, 141, 141, 141, 141]


for i in range(0, len(lstema0)):
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ta-sma.{}'.format(lstema0[i])],
    )

    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[lstema0[i]],
    )

    buy2 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['up'],
        strVals=['ta-sma.{}'.format(lstema1[i])],
    )

    buy3 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[lstema1[i]],
    )        
    
    buy4 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ta-sma.{}'.format(lstema1[i])],
        group=1,
    )

    buy5 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[lstema0[i]],
        group=1,        
    )

    buy6 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['up'],
        strVals=['ta-sma.{}'.format(lstema0[i])],
        group=1,        
    )

    buy7 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[lstema1[i]],
        group=1,        
    )            

    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ta-sma.{}'.format(lstema0[i])],
    )
    
    sell1 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ta-sma.{}'.format(lstema1[i])],
        group=1,                
    )    

#         sell1 = trdb2py.trading2_pb2.CtrlCondition(
#             name='indicatorsp',
#             operators=['down'],
#             strVals=['ta-sma.{}'.format(ema0)],
#         )

    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0, buy1, buy2, buy3, buy4, buy5, buy6, buy7])
    s0.sell.extend([sell0, sell1])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='sma{}&sma{}'.format(lstema0[i], lstema1[i]),
    ))
        
lstpnlmixF = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=3)        

trdb2py.showPNLs(lstpnlmixF0 + lstpnlmixF + [pnlBaselineF], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmixF + lstpnlmixF0 + [pnlBaselineF])

# dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 2]

dfpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)